<a href="https://colab.research.google.com/github/diogosantanaime/rec_proc/blob/main/run_flair_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação da biblioteca 'flair' usando o pip
!pip install flair


  Using cached flair-0.12.2-py3-none-any.whl (373 kB)
  Using cached segtok-1.5.11-py3-none-any.whl (24 kB)
  Using cached mpld3-0.3-py3-none-any.whl
  Using cached sqlitedict-2.1.0-py3-none-any.whl
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached boto3-1.26.163-py3-none-any.whl (135 kB)
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)
  Using cached gdown-4.4.0-py3-none-any.whl
  Using cached conllu-4.5.3-py2.py3-none-any.whl (16 kB)
  Using cached Wikipedia_API-0.5.8-py3-none-any.whl (13 kB)
  Using cached pptree-3.1-py3-none-any.whl
  Using cached pytorch_revgrad-0.2.0-py3-none-any.whl (4.6 kB)
  Using cached transformer_smaller_training_vocab-0.2.4-py3-none-any.whl (13 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17

In [ ]:
import os
import requests
import matplotlib.pyplot as plt


from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings, FlairEmbeddings, TransformerWordEmbeddings
from typing import List
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.optim import SGDW


In [ ]:
# Diretório onde estão os arquivos de treinamento, teste e validação
data_folder = "/content/drive/MyDrive/dataset"

columns = {0: "text", 1: "ner"}

# Carrega os dados do PET Dataset em um objeto Corpus do Flair
pet = ColumnCorpus(data_folder, columns,
                      train_file="train_pet.txt",
                      test_file="test_pet.txt",
                      dev_file="val_pet.txt")

# Carrega os dados do Wikineural Dataset em um objeto Corpus do Flair
wikineural = ColumnCorpus(data_folder, columns,
                      train_file="train_wikineural.txt",
                      test_file="test_wikineural.txt",
                      dev_file="val_wikineural.txt")

# Carrega os dados do conll03 Dataset em um objeto Corpus do Flair
conll03 = ColumnCorpus(data_folder, columns,
                      train_file="train_conll03.txt",
                      test_file="test_conll03.txt",
                      dev_file="val_conll03.txt")

# Define a lista de corpus para o experimento
corpus_dataset = [pet, wikineural, conll03]

# Define o tipo de tag
tag_type = 'ner'

2023-06-29 16:23:07,564 Reading data from /content/drive/MyDrive/dataset
2023-06-29 16:23:07,566 Train: /content/drive/MyDrive/dataset/train_pet.txt
2023-06-29 16:23:07,567 Dev: /content/drive/MyDrive/dataset/val_pet.txt
2023-06-29 16:23:07,568 Test: /content/drive/MyDrive/dataset/test_pet.txt
2023-06-29 16:23:10,083 Reading data from /content/drive/MyDrive/dataset
2023-06-29 16:23:10,085 Train: /content/drive/MyDrive/dataset/train_wikineural.txt
2023-06-29 16:23:10,088 Dev: /content/drive/MyDrive/dataset/val_wikineural.txt
2023-06-29 16:23:10,090 Test: /content/drive/MyDrive/dataset/test_wikineural.txt
2023-06-29 16:23:19,456 Reading data from /content/drive/MyDrive/dataset
2023-06-29 16:23:19,458 Train: /content/drive/MyDrive/dataset/train_conll03.txt
2023-06-29 16:23:19,459 Dev: /content/drive/MyDrive/dataset/val_conll03.txt
2023-06-29 16:23:19,461 Test: /content/drive/MyDrive/dataset/test_conll03.txt


bi-LSTM

In [ ]:
for corpus in corpus_dataset:
    # Cria um dicionário de tags usando o corpus combinado
    tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

    # Define o tipo de embeddings desejado
    embedding_types: List[TokenEmbeddings] = [
        FlairEmbeddings('news-forward'),
        FlairEmbeddings('news-backward'),
    ]

    # Cria as embeddings empilhadas
    embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

    # Criação de um objeto SequenceTagger
    tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,                   # Tamanho da camada oculta do modelo
        embeddings=embeddings,             # Embeddings utilizados como entrada para o modelo
        tag_dictionary=tag_dictionary,     # Dicionário de rótulos utilizado pelo modelo
        tag_type=tag_type,                 # Tipo de rótulo utilizado (por exemplo, NER)
        use_crf=True                      # Indicação de uso do CRF (Conditional Random Field) para decodificar as sequências de rótulos
    )

    # Treino do modelo Flair
    trainer: ModelTrainer = ModelTrainer(tagger, corpus)
    trainer.train(
        f'resources/biLSTM/taggers/{corpus}',   # Diretório para salvar os resultados do treinamento
        learning_rate=0.1,                      # Taxa de aprendizado
        mini_batch_size=32,                     # Tamanho do mini-batch
        optimizer=SGDW,                         # Otimizador
        max_epochs=10                           # Número máximo de épocas
    )


2023-06-29 12:21:27,363 Computing label dictionary. Progress:


333it [00:00, 34756.83it/s]

2023-06-29 12:21:27,408 Dictionary created for label 'ner' with 8 values: ActivityData (seen 402 times), Activity (seen 400 times), Actor (seen 373 times), XORGateway (seen 93 times), ConditionSpecification (seen 72 times), FurtherSpecification (seen 60 times), ANDGateway (seen 6 times)


2023-06-29 12:21:28,748 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmp023p4wki


100%|██████████| 69.7M/69.7M [00:06<00:00, 10.6MB/s]

...

100%|██████████| 102/102 [00:13<00:00,  7.56it/s]

2023-06-29 12:50:40,078 Evaluating as a multi-label problem: False
2023-06-29 12:50:40,128 DEV : loss 0.03218387812376022 - f1-score (micro avg)  0.9441


2023-06-29 12:50:40,310 BAD EPOCHS (no improvement): 0
2023-06-29 12:50:40,312 saving best model
2023-06-29 12:50:41,897 ----------------------------------------------------------------------------------------------------
2023-06-29 12:50:42,699 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>


100%|██████████| 108/108 [00:33<00:00,  3.21it/s]

2023-06-29 12:51:16,687 Evaluating as a multi-label problem: False
2023-06-29 12:51:16,736 0.9106	0.9017	0.9061	0.8678
2023-06-29 12:51:16,737 
Results:
- F-score (micro) 0.9061
- F-score (macro) 0.8924
- Accuracy 0.8678

By class:
              precision    recall  f1-score   support

         LOC     0.9284    0.9167    0.9225      1668
         ORG     0.8844    0.8754    0.8799      1661
         PER     0.9550    0.9592    0.9571      1617
        MISC     0.8245    0.7963    0.8101       702

   micro avg     0.9106    0.9017    0.9061      5648
   macro avg     0.8981    0.8869    0.8924      5648
weighted avg     0.9102    0.9017    0.9059      5648

2023-06-29 12:51:16,740 ----------------------------------------------------------------------------------------------------


Glove

In [ ]:
for corpus in corpus_dataset:
    # Cria um dicionário de tags usando o corpus combinado
    tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

    # Define o tipo de embedding desejado
    embeddings = WordEmbeddings('glove')

    # Criação de um objeto SequenceTagger
    tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,                   # Tamanho da camada oculta do modelo
        embeddings=embeddings,             # Embeddings utilizados como entrada para o modelo
        tag_dictionary=tag_dictionary,     # Dicionário de rótulos utilizado pelo modelo
        tag_type=tag_type,                 # Tipo de rótulo utilizado (por exemplo, NER)
        use_crf=True                      # Indicação de uso do CRF (Conditional Random Field) para decodificar as sequências de rótulos
    )

    # Treino do modelo Flair
    trainer: ModelTrainer = ModelTrainer(tagger, corpus)
    trainer.train(
        f'resources/glove/taggers/{corpus}',   # Diretório para salvar os resultados do treinamento
        learning_rate=0.1,                      # Taxa de aprendizado
        mini_batch_size=32,                     # Tamanho do mini-batch
        optimizer=SGDW,                         # Otimizador
        max_epochs=10                           # Número máximo de épocas
    )


2023-06-29 12:51:16,751 Computing label dictionary. Progress:


333it [00:00, 31288.15it/s]

2023-06-29 12:51:16,768 Dictionary created for label 'ner' with 8 values: ActivityData (seen 402 times), Activity (seen 400 times), Actor (seen 373 times), XORGateway (seen 93 times), ConditionSpecification (seen 72 times), FurtherSpecification (seen 60 times), ANDGateway (seen 6 times)


2023-06-29 12:51:17,780 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpv6h96aiv


100%|██████████| 153M/153M [00:13<00:00, 12.0MB/s]

...

100%|██████████| 102/102 [00:11<00:00,  9.04it/s]

2023-06-29 13:09:07,350 Evaluating as a multi-label problem: False
2023-06-29 13:09:07,412 DEV : loss 0.08322510123252869 - f1-score (micro avg)  0.8761


2023-06-29 13:09:07,655 BAD EPOCHS (no improvement): 1
2023-06-29 13:09:08,324 ----------------------------------------------------------------------------------------------------
2023-06-29 13:09:08,868 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>


100%|██████████| 108/108 [00:10<00:00, 10.79it/s]

2023-06-29 13:09:18,992 Evaluating as a multi-label problem: False
2023-06-29 13:09:19,050 0.8769	0.815	0.8448	0.7529
2023-06-29 13:09:19,052 
Results:
- F-score (micro) 0.8448
- F-score (macro) 0.8245
- Accuracy 0.7529

By class:
              precision    recall  f1-score   support

         LOC     0.8418    0.8993    0.8696      1668
         PER     0.9337    0.8967    0.9148      1617
         ORG     0.8833    0.7243    0.7959      1661
        MISC     0.8152    0.6410    0.7177       702

   micro avg     0.8769    0.8150    0.8448      5648
   macro avg     0.8685    0.7903    0.8245      5648
weighted avg     0.8770    0.8150    0.8420      5648

2023-06-29 13:09:19,053 ----------------------------------------------------------------------------------------------------


RoBERTa

In [ ]:
for corpus in corpus_dataset:
    # Cria um dicionário de tags usando o corpus combinado
    tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

    # Define o tipo de embedding desejado
    embeddings = TransformerWordEmbeddings("roberta-base", layers="all", layer_mean=True)

    # Criação de um objeto SequenceTagger
    tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,                   # Tamanho da camada oculta do modelo
        embeddings=embeddings,             # Embeddings utilizados como entrada para o modelo
        tag_dictionary=tag_dictionary,     # Dicionário de rótulos utilizado pelo modelo
        tag_type=tag_type,                 # Tipo de rótulo utilizado (por exemplo, NER)
        use_crf=True                      # Indicação de uso do CRF (Conditional Random Field) para decodificar as sequências de rótulos
    )

    # Treino do modelo Flair
    trainer: ModelTrainer = ModelTrainer(tagger, corpus)
    trainer.train(
        f'resources/roberta/taggers/{corpus}',    # Diretório para salvar os resultados do treinamento
        learning_rate=0.1,                        # Taxa de aprendizado
        mini_batch_size=32,                       # Tamanho do mini-batch
        optimizer=SGDW,                           # Otimizador
        max_epochs=10                             # Número máximo de épocas
    )


2023-06-29 13:09:19,068 Computing label dictionary. Progress:


333it [00:00, 29406.14it/s]

2023-06-29 13:09:19,087 Dictionary created for label 'ner' with 8 values: ActivityData (seen 402 times), Activity (seen 400 times), Actor (seen 373 times), XORGateway (seen 93 times), ConditionSpecification (seen 72 times), FurtherSpecification (seen 60 times), ANDGateway (seen 6 times)


2023-06-29 13:09:27,848 SequenceTagger predicts: Dictionary with 29 tags: O, S-ActivityData, B-ActivityData, E-ActivityData, I-ActivityData, S-Activity, B-Activity, E-Activity, I-Activity, S-Actor, B-Actor, E-Actor, I-Actor, S-XORGateway, B-XORGateway, E-XORGateway, I-XORGateway, S-ConditionSpecification, B-ConditionSpecification, E-ConditionSpecification, I-ConditionSpecification, S-FurtherSpecification, B-FurtherSpecification, E-FurtherSpecification, I-FurtherSpecification, S-ANDGateway, B-ANDGateway, E-ANDGateway, I-ANDGateway
2023-06-29 13:09:27,877 ----------------------------------------------------------------------------------------------------
2023-06-29 13:09:27,879 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNo

100%|██████████| 2/2 [00:00<00:00,  7.68it/s]

...

2023-06-29 14:28:52,304 Evaluating as a multi-label problem: False


2023-06-29 14:28:52,363 DEV : loss 0.0781949907541275 - f1-score (micro avg)  0.9361
2023-06-29 14:28:52,785 BAD EPOCHS (no improvement): 1
2023-06-29 14:28:54,734 ----------------------------------------------------------------------------------------------------
2023-06-29 14:28:57,340 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>


100%|██████████| 108/108 [00:20<00:00,  5.31it/s]

2023-06-29 14:29:17,915 Evaluating as a multi-label problem: False


2023-06-29 14:29:17,966 0.8899	0.903	0.8964	0.8528
2023-06-29 14:29:17,967 
Results:
- F-score (micro) 0.8964
- F-score (macro) 0.8837
- Accuracy 0.8528

By class:
              precision    recall  f1-score   support

         ORG     0.8404    0.8784    0.8590      1661
         LOC     0.9243    0.9221    0.9232      1668
         PER     0.9424    0.9518    0.9471      1617
        MISC     0.8080    0.8034    0.8057       702

   micro avg     0.8899    0.9030    0.8964      5648
   macro avg     0.8788    0.8889    0.8837      5648
weighted avg     0.8904    0.9030    0.8965      5648

2023-06-29 14:29:17,969 ----------------------------------------------------------------------------------------------------


BERT Base

In [ ]:
for corpus in corpus_dataset:
    # Cria um dicionário de tags usando o corpus combinado
    tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

    # Define o tipo de embedding desejado
    embeddings = TransformerWordEmbeddings(
        "bert-base-uncased",           # Modelo BERT-base pré-treinado
        fine_tune=False,                # Não permite ajuste fino dos pesos durante o treinamento
        layers='-1,-2,-3,-4',           # Utiliza as últimas 4 camadas do modelo BERT
        allow_long_sentences=True       # Permite o processamento de frases longas
    )

    # Criação de um objeto SequenceTagger
    tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,                   # Tamanho da camada oculta do modelo
        embeddings=embeddings,             # Embeddings utilizados como entrada para o modelo
        tag_dictionary=tag_dictionary,     # Dicionário de rótulos utilizado pelo modelo
        tag_type=tag_type,                 # Tipo de rótulo utilizado (por exemplo, NER)
        use_crf=False                      # Indicação de uso do CRF (Conditional Random Field) para decodificar as sequências de rótulos
    )

    # Treino do modelo Flair
    trainer: ModelTrainer = ModelTrainer(tagger, corpus)
    trainer.train(
        f'resources/bert-base/taggers/{corpus}',    # Diretório para salvar os resultados do treinamento
        learning_rate=0.1,                      # Taxa de aprendizado
        mini_batch_size=16,                     # Tamanho do mini-batch
        optimizer=SGDW,                         # Otimizador
        max_epochs=10                           # Número máximo de épocas
    )

2023-06-29 14:29:17,981 Computing label dictionary. Progress:


333it [00:00, 27571.23it/s]

2023-06-29 14:29:17,999 Dictionary created for label 'ner' with 8 values: ActivityData (seen 402 times), Activity (seen 400 times), Actor (seen 373 times), XORGateway (seen 93 times), ConditionSpecification (seen 72 times), FurtherSpecification (seen 60 times), ANDGateway (seen 6 times)


2023-06-29 14:29:24,725 SequenceTagger predicts: Dictionary with 29 tags: O, S-ActivityData, B-ActivityData, E-ActivityData, I-ActivityData, S-Activity, B-Activity, E-Activity, I-Activity, S-Actor, B-Actor, E-Actor, I-Actor, S-XORGateway, B-XORGateway, E-XORGateway, I-XORGateway, S-ConditionSpecification, B-ConditionSpecification, E-ConditionSpecification, I-ConditionSpecification, S-FurtherSpecification, B-FurtherSpecification, E-FurtherSpecification, I-FurtherSpecification, S-ANDGateway, B-ANDGateway, E-ANDGateway, I-ANDGateway
2023-06-29 14:29:24,752 ----------------------------------------------------------------------------------------------------
2023-06-29 14:29:24,755 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,)

100%|██████████| 3/3 [00:00<00:00, 15.80it/s]

...

2023-06-29 14:42:51,003 Evaluating as a multi-label problem: False
2023-06-29 14:42:51,058 DEV : loss 0.06640505790710449 - f1-score (micro avg)  0.9216


2023-06-29 14:42:51,354 BAD EPOCHS (no improvement): 0
2023-06-29 14:42:51,356 saving best model
2023-06-29 14:44:32,263 ----------------------------------------------------------------------------------------------------
2023-06-29 14:44:38,385 SequenceTagger predicts: Dictionary with 17 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC


100%|██████████| 216/216 [00:12<00:00, 17.31it/s]

2023-06-29 14:44:51,164 Evaluating as a multi-label problem: False


2023-06-29 14:44:51,249 0.882	0.8943	0.8881	0.8335
2023-06-29 14:44:51,251 
Results:
- F-score (micro) 0.8881
- F-score (macro) 0.8723
- Accuracy 0.8335

By class:
              precision    recall  f1-score   support

         ORG     0.8279    0.8688    0.8478      1661
         LOC     0.9115    0.9017    0.9066      1668
         PER     0.9485    0.9691    0.9587      1617
        MISC     0.7874    0.7650    0.7760       702

   micro avg     0.8820    0.8943    0.8881      5648
   macro avg     0.8688    0.8761    0.8723      5648
weighted avg     0.8821    0.8943    0.8880      5648

2023-06-29 14:44:51,252 ----------------------------------------------------------------------------------------------------


BERT Large

In [ ]:
for corpus in corpus_dataset:
    # Cria um dicionário de tags usando o corpus combinado
    tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

    # Define o tipo de embedding desejado
    # Define o tipo de embedding desejado
    embeddings = TransformerWordEmbeddings(
        "bert-large-uncased",           # Modelo BERT-large pré-treinado
        fine_tune=False,                # Não permite ajuste fino dos pesos durante o treinamento
        layers='-1,-2,-3,-4',           # Utiliza as últimas 4 camadas do modelo BERT
        allow_long_sentences=True       # Permite o processamento de frases longas
    )


    # Criação de um objeto SequenceTagger
    tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,                   # Tamanho da camada oculta do modelo
        embeddings=embeddings,             # Embeddings utilizados como entrada para o modelo
        tag_dictionary=tag_dictionary,     # Dicionário de rótulos utilizado pelo modelo
        tag_type=tag_type,                 # Tipo de rótulo utilizado (por exemplo, NER)
        use_crf=True                      # Indicação de uso do CRF (Conditional Random Field) para decodificar as sequências de rótulos
    )

    # Treino do modelo Flair
    trainer: ModelTrainer = ModelTrainer(tagger, corpus)
    trainer.train(
        f'resources/bert-large/taggers/{corpus}',    # Diretório para salvar os resultados do treinamento
        learning_rate=0.1,                      # Taxa de aprendizado
        mini_batch_size=16,                     # Tamanho do mini-batch
        optimizer=SGDW,                         # Otimizador
        max_epochs=10                           # Número máximo de épocas
    )

2023-06-29 16:23:26,819 Computing label dictionary. Progress:


333it [00:00, 22855.19it/s]

2023-06-29 16:23:26,869 Dictionary created for label 'ner' with 8 values: ActivityData (seen 402 times), Activity (seen 400 times), Actor (seen 373 times), XORGateway (seen 93 times), ConditionSpecification (seen 72 times), FurtherSpecification (seen 60 times), ANDGateway (seen 6 times)


2023-06-29 16:23:46,621 SequenceTagger predicts: Dictionary with 29 tags: O, S-ActivityData, B-ActivityData, E-ActivityData, I-ActivityData, S-Activity, B-Activity, E-Activity, I-Activity, S-Actor, B-Actor, E-Actor, I-Actor, S-XORGateway, B-XORGateway, E-XORGateway, I-XORGateway, S-ConditionSpecification, B-ConditionSpecification, E-ConditionSpecification, I-ConditionSpecification, S-FurtherSpecification, B-FurtherSpecification, E-FurtherSpecification, I-FurtherSpecification, S-ANDGateway, B-ANDGateway, E-ANDGateway, I-ANDGateway
2023-06-29 16:23:49,302 ----------------------------------------------------------------------------------------------------
2023-06-29 16:23:49,306 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30523, 1024)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((10

/usr/local/lib/python3.10/dist-packages/flair/optim.py:132: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  p.data.add_(-group["lr"], d_p)


...

100%|██████████| 204/204 [00:11<00:00, 18.17it/s]

2023-06-29 16:54:08,836 Evaluating as a multi-label problem: False
2023-06-29 16:54:08,888 DEV : loss 0.04870393127202988 - f1-score (micro avg)  0.9231


2023-06-29 16:54:09,012 BAD EPOCHS (no improvement): 0
2023-06-29 16:54:09,014 saving best model
2023-06-29 16:54:26,286 ----------------------------------------------------------------------------------------------------
2023-06-29 16:54:30,655 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>


100%|██████████| 216/216 [00:36<00:00,  5.90it/s]

2023-06-29 16:55:07,673 Evaluating as a multi-label problem: False
2023-06-29 16:55:07,722 0.8892	0.8814	0.8853	0.8288
2023-06-29 16:55:07,724 
Results:
- F-score (micro) 0.8853
- F-score (macro) 0.8679
- Accuracy 0.8288

By class:
              precision    recall  f1-score   support

         LOC     0.8884    0.9119    0.9000      1668
         ORG     0.8687    0.8447    0.8565      1661
         PER     0.9639    0.9419    0.9528      1617
        MISC     0.7685    0.7564    0.7624       702

   micro avg     0.8892    0.8814    0.8853      5648
   macro avg     0.8724    0.8637    0.8679      5648
weighted avg     0.8893    0.8814    0.8852      5648

2023-06-29 16:55:07,725 ----------------------------------------------------------------------------------------------------
